In [1]:
import ee 
import os
import geemap

ee.Initialize()


LandSat8 data

In [2]:
# US lower 48 states aoi
lower48 = ee.FeatureCollection("USDOS/LSIB_SIMPLE/2017").filter(ee.Filter.eq('country_na', 'United States')).geometry().dissolve()

In [3]:
# function to mask out cloud & cloud shadows
def maskL8SR(image):
  cloudShadowBitMask = ee.Number(2).pow(3).int()
  cloudsBitMask = ee.Number(2).pow(5).int()
  qa = image.select('QA_PIXEL')
  mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0).And(qa.bitwiseAnd(cloudsBitMask).eq(0))
  return image.updateMask(mask)

In [4]:
# function to compute NDVI
def getNDVI(image):
    return image.normalizedDifference(['SR_B5', 'SR_B4'])

In [5]:
# function to compute NDMI
def getNDMI(image):
    return image.normalizedDifference(['SR_B5', 'SR_B6'])

In [6]:
# pull LandSat8 data filtered by date, lower48 aoi, and mask out cloud & cloud shadow
ls8 = (ee.ImageCollection('LANDSAT/LC08/C02/T1_L2')
    .filterDate('2021-9-01', '2021-11-01')
    .filterBounds(lower48)
    .map(maskL8SR))

In [45]:
# compute NDVI
ls8ndvi = ls8.map(getNDVI)
# create mean composite image
ls8ndvimean = ls8ndvi.mean()
# create mask for NDVI values >0.2 (***Carl's workflow did this for values >0.4 but >0.25 seemed to capture more truly irrigated areas***)
ndviMask = ls8ndvimean.updateMask(ls8ndvimean.gte(0.25))

In [46]:
# compute NDMI
ls8ndmi = ls8.map(getNDMI)
# create mean composite image
ls8ndmimean = ls8ndmi.mean()
# create mask for NDMI values >0.05 (***Carl's workflow did this for values >0.13 but >0.10 seemed to capture more truly irrigated areas***)
ndmiMask = ls8ndmimean.updateMask(ls8ndmimean.gte(0.10))

In [47]:
# Create NDVI & NDMI mosaic mask
ndvi_ndmi_mosaic = ndmiMask.And(ndviMask)

SIG WUI-non-WUI raster

In [48]:
# read in wui raster
wui = ee.Image('projects/pyregence-ee/assets/conus/vulnerability/wui_v3_w_nonWUIclasses')

In [49]:
# mask out wildland
wuimask = wui.updateMask(wui.lte(4))

In [50]:
# replace where ndvi_ndmi mask & wui overlap with new value (5)
wuitest = ndvi_ndmi_mosaic.where(wuimask.select('wui_class').gte(0), 5)

In [51]:
# extract ndvi_ndmi wui overlap areas
wui_index = wuitest.updateMask(wuitest.gte(5))

In [52]:
# calling the fbfm raster from the cloud
fm40 = ee.Image("projects/pyregence-ee/assets/conus/fuels/Fuels_FM40_2021_12")

In [53]:
# mask where fbfm is grass or grass-shrub
fm40_grass_target = fm40.gte(101).And(fm40.lte(124)) # GR & GS fuel models

In [54]:
#mask where wui_index and gr/gs overlap
finalmask = wui_index.updateMask(fm40_grass_target)

In [57]:
# Reduce finalmask to only areas gte 0.5 acre (~2000 sq m)
minArea = 2000
maxSize = 200
pixelCount = finalmask.connectedPixelCount(maxSize)
minPixelCount = ee.Image(minArea).divide(ee.Image.pixelArea())
finalmask = finalmask.updateMask(pixelCount.gte(minPixelCount))

In [60]:
# where finalmask, replace fbfm with NB fuel model 93 
fm40_NB_replace = fm40.where(finalmask,93) # 93 is NB Agriculture

In [61]:
Map=geemap.Map()
#Map.addLayer(ndvi_ndmi_mosaic, {'palette': 'red'}, 'ndvi_ndmi mask')
#Map.addLayer(wui_index, {'palette': 'purple'}, 'wui_index mask')
Map.addLayer(finalmask, {'palette': 'black'}, 'final mask')
Map.addLayer(fm40, {'min':91,'max':204,'palette':['grey','yellow','orange','brown','green','blue','purple']},'FM40')
#Map.addLayer(fm40_grass_target.selfMask(), {'min':0,'max':1,'palette':['yellow']},'fm40 grass target',False)
Map.addLayer(fm40_NB_replace, {'min':91,'max':204,'palette':['grey','yellow','orange','brown','green','blue','purple']},'fm40 replace grass w NonBurnable')
# Map.addLayer(wui.select('wui_class'),{'min':1,'max':6, 'palette':['yellow','orange','red','grey','green']}, 'WUI w Non-WUI Urban and Wildland')
# Map.addLayer(wuimask.select('wui_class'),{'min':1,'max':5, 'palette':['yellow','orange','red','grey']}, 'WUI mask')
Map.setCenter(-118.03,33.929,13)

Map


Map(center=[33.929, -118.03], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(chil…

In [20]:
# task = ee.batch.Export.image.toAsset(
#     image= fm40_NB_replace,
#     description= 'export irrigation replacements FM40',
#     assetId= f'projects/pyregence-ee/assets/conus/fuels/Fuels_FM40_irrigation_replacement_test',
#     region = fm40_NB_replace.geometry(),
#     crs= 'EPSG:5070', 
#     maxPixels= 1e13,
#     pyramidingPolicy = {".default":"mode"}
#     )
# task.start()